In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
import itertools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import warnings

warnings.filterwarnings('ignore')

train_set = pd.read_csv('train_set.csv')
test_set = pd.read_csv('test_set.csv')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Split the training dataset into features (x) and class (y)
x_train = train_set.copy()
x_train = train_set.drop("Class", axis=1)
y_train = train_set["Class"]

# Split the training dataset into features (x) and class (y)
x_test = test_set.copy()
x_test = x_test.drop("Class", axis=1)
y_test = test_set["Class"]

display(x_train.head())

train_len = len(y_train)
test_len = len(y_test)
total_len = train_len + test_len
print(total_len)
print(f"Number of training dataset records: {train_len}, {'{:.2f}%'.format(100 * train_len / total_len)} of total")
print(f"Number of test dataset records: {len(x_test)}, {'{:.2f}%'.format(100 * test_len / total_len)} of total")

,LowestVE/VCO2,PeakVO2Real,DiffPeakVO2,DiffPercentPeakVO2,MeanVO2,PeakVO2,MeanVCO2,O2PulseDiff,O2PulsePercent,VO2vsPeakVO2atVT
0,0.125760,-0.418146,0.363240,-0.858639,-0.015964,-0.418146,-0.312581,-0.702406,-0.574696,0.077734
1,0.638246,-1.135581,-0.584887,-0.521123,-1.149392,-1.135581,-1.196624,0.031928,-1.404812,-1.043617
2,-0.668593,-0.436081,-0.199208,-0.408618,-0.510841,-0.436081,-0.441169,-0.399094,-0.878007,-0.082459
3,-1.667941,0.012315,-0.954495,0.073547,-0.223493,0.012315,-0.071479,-0.175601,-1.037645,-0.146536
4,0.689494,-0.310530,0.845338,-1.501526,0.782225,-0.310530,1.117960,-0.654515,-0.670478,-0.643134


202
Number of training dataset records: 136, 67.33% of total
Number of test dataset records: 66, 32.67% of total


In [ ]:
print(x_train.columns, len(x_train.columns))

Index(['LowestVE/VCO2', 'PeakVO2Real', 'DiffPeakVO2', 'DiffPercentPeakVO2',
       'MeanVO2', 'PeakVO2', 'MeanVCO2', 'O2PulseDiff', 'O2PulsePercent',
       'VO2vsPeakVO2atVT'],
      dtype='object') 10


In [ ]:
# Decision Tree Classifier

clf = DecisionTreeClassifier()
hyperparams = {
  "criterion": ["gini", "entropy", "log_loss"],
  "splitter": ["best", "random"],
  "max_depth": [None, 2, 4, 6, 8],
  "min_samples_split": [2, 4, 6, 8, 10],
  "min_samples_leaf": [1, 2, 4, 6, 8],
  "max_features": [None, "log2", "sqrt"],
  "class_weight": [None, "balanced"]
}
grid_search = GridSearchCV(clf, hyperparams, cv=5)
grid_search.fit(x_train, y_train)
print(grid_search.best_params_)

accuracies = []
precisions = []
recalls = []
f1s = []
for i in range(1000):
  y_pred = grid_search.predict(x_test)

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  f1 = f1_score(y_test, y_pred, average='weighted')

  accuracies.append(accuracy)
  precisions.append(precision)
  recalls.append(recall)
  f1s.append(f1)

{'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'}


In [ ]:
ave_accuracy = np.mean(accuracies)
ave_precision = np.mean(precisions)
ave_recall = np.mean(recalls)
ave_f1 = np.mean(f1s)

print("Average Scores:")
print("Accuracy:", ave_accuracy)
print("Precision:", ave_precision)
print("Recall:", ave_recall)
print("F1 Score:", ave_f1)

Average Scores:
Accuracy: 0.5151515151515154
Precision: 0.5828930328930328
Recall: 0.5151515151515154
F1 Score: 0.5265170910332201
